In [ ]:
import csv
import os 
import sys 
import xnat


#TO EDIT:
#Do not include '/' after the .org url e.g use 'https://demo.xnat.org' and NOT 'https://demo.xnat.org/'
xnatURL="https://oxi.circ.wustl.edu"
user="username"
password="password"

project = "ProjectName"
#path where you want the data to be downloaded to (it will be located in a folder called 'XNATDownload' under this path)
pathToDownloadData= "/path/to/download/to"
#the path to the csv is the one downloaded from xnat 
pathToCSV = "/path/to/assessment/csv/"
#Set below to true if you would like the associated jupyternotebook jpg downloaded as well. 
downloadJupyterJPG=False


#DONT EDIT BELOW THIS
downloadPath = os.path.join(pathToDownloadData, "XNATDownload")
if not os.path.exists(downloadPath):
    os.mkdir(downloadPath)


session = xnat.connect(xnatURL, user, password)


with open(pathToCSV, newline='') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvReader, None)
    next(csvReader, None)
    for row in csvReader:
        # print(row[0])
        assessorInfo = row[0].split("_")
        subject = assessorInfo[0]
        sesssion = "_".join(assessorInfo[0:2])
        assessor = row[0]

        #download notebook if user wants 
        if downloadJupyterJPG:
            notebookJPGPath="{xnatURL}/data/projects/{projectName}/subjects/{subjectLabel}/experiments/{sessionLabel}/assessors/{assessorLabel}/resources/RESULTS/files/images/output.jpg".format(xnatURL=xnatURL,projectName=project, subjectLabel=subject, sessionLabel=sesssion,assessorLabel=assessor)

            #try and download the notebook if there is one
            try:
                print("Downloading notebook, ", notebookJPGPath)
                response = session.get(notebookJPGPath)
                fileName=assessor+".jpg"
                path=os.path.join(downloadPath, fileName)
                with open(path, 'wb') as f:
                    f.write(response.content)
            except: 
                print("ERROR: No notebook at ", notebookJPGPath)

        #if you are downloading Reconstruction data or Full-Processing data then the mat file will be downloaded
        if "Reconstruction" in assessorInfo or "Full-Processing" in assessorInfo:
            scan=assessorInfo[2]
            matFilePath="{xnatURL}/data/projects/{projectName}/subjects/{subjectLabel}/experiments/{sessionLabel}/scans/{scanLabel}/resources/FNIRS/files".format(xnatURL=xnatURL,projectName=project, subjectLabel=subject, sessionLabel=sesssion,scanLabel=scan)
            response = session.get(matFilePath+"?format=json")
            FoundFile=False
            for file in response.json()['ResultSet']['Result']:
                xnatFile = file["Name"]
                if 'params' in xnatFile and '.mat' in xnatFile:
                    fileNameComponents = xnatFile.split("_")
                    fileNameComponents.insert(2,'scan-01')
                    finalFileName="_".join(fileNameComponents)
                    path=os.path.join(downloadPath, finalFileName)
                    print("Downloading mat file", xnatFile)
                    session.download(matFilePath+"/"+xnatFile, path, verbose=False)
                    # path=os.path.join(downloadPath, fileName)
                    # with open(path, 'wb') as f:
                    #     for chunk in response.iter_content(chunk_size=1024*1024):
                    #         if chunk:
                    #             f.write(chunk)
                    FoundFile=True
            if not FoundFile:
                print("No matching roconstruction '.mat' found for ", assessor)